In [1]:
import pyfits
from scipy.ndimage import gaussian_filter
import scipy.optimize as opt
import numpy as np
import pylab as plt
import rydlab
import pandas as pd
import os
import copy
from lmfit import Model
from lmfit.models import LorentzianModel
from lmfit.models import ExponentialModel
from lmfit.models import ConstantModel
from lmfit import Parameters, minimize, report_fit, Model
from matplotlib.colors import LinearSegmentedColormap, to_rgb
import seaborn as sns
sns.set_style("dark")
#sns.set_style("darkgrid")
import matplotlib as mpl

pd.options.display.max_colwidth = 120
mpl.rc('image', cmap='afmhot')
path= 'A:/Projekte - Projects/2019_IEI/July/08'
file_date = '2019-07-08'
folders = rydlab.analyze_folder(path,filter=False) 
slicer = (slice(10,110),slice(10,320))
binning_scale = 1
folders

In [2]:
path=folders.Name[7]

os.chdir(path)

variables = np.loadtxt( '2019-07-08_variables.dat' )[:,1]

N = len(variables)

print(os.getcwd())

def params_to_dict(params):
    return {key: params[key].value for key in params.keys()}


class Fit2d:
    colors = [(1, 1, 1), to_rgb("#5597C6"), to_rgb("#60BD68"), to_rgb("#FAA43A"), to_rgb("#d37096")]

    c_map_name = 'my_list'
    cm = LinearSegmentedColormap.from_list(c_map_name, colors, N=200)

    @staticmethod
    def _function(x, y):
        pass
    
    def __init__(self, data, x=None, y=None, params=None):
        self.data = data
        if x is None or y is None:
            self.x, self.y = self.get_mesh()

        if params is None:
            model = Model(self._function)
            params = model.make_params()

        self.params = params
        self.fit_object = None

    def fit_data(self, method='LeastSq'):
        fit_object = minimize(self.residuals, self.params, method=method)
        self.fit_object = fit_object
        self.params = fit_object.params

    def residuals(self, p):
        return self.data - self._function([self.x, self.y], **params_to_dict(p))

    def plot(self, ax):
        ax.imshow(self.data, cmap=self.cm)
        ax.contour(self.x, self.y, self._function([self.x, self.y], **params_to_dict(self.params)),
                   5, colors='w', linewidths=0.5)

    def get_mesh(self):
        x = np.arange(0, np.shape(self.data)[1], 1)
        y = np.arange(0, np.shape(self.data)[0], 1)
        return np.meshgrid(x, y)

    def report(self):
        print(report_fit(self.fit_object))


class Fit2dGaussian(Fit2d):
    def __init__(self, data, x=None, y=None, params=None):
        super().__init__(data, x, y, params)

    @staticmethod
    def _function(args, amp=1, cen_x=250, cen_y=50, sig_x=50, sig_y=10, offset=0):
        x, y = args
        return amp * np.exp(-(((cen_x - x) / sig_x) ** 2 + ((cen_y - y) / sig_y) ** 2) / 2.0) + offset


class Fit2d2Gaussian(Fit2d):
    def __init__(self, data, x=None, y=None, params=None):
        super().__init__(data, x, y, params)

    @staticmethod
    def _function(args, amp=1, cen_x=250, cen_y=50, sig_x=50, sig_y=10,
                  amp2=1, cen_x2=250, cen_y2=50, sig_x2=50, sig_y2=10,
                  offset=0):
        x, y = args
        return (amp * np.exp(-(((cen_x - x) / sig_x) ** 2 + ((cen_y - y) / sig_y) ** 2) / 2.0)
                + amp2 * np.exp(-(((cen_x2 - x) / sig_x2) ** 2 + ((cen_y2 - y) / sig_y2) ** 2) / 2.0)
                + offset)


def fitsopen(n,bg):
    if n<10:
        hdulist = pyfits.open(file_date+str("_")+str(0)+str(n)+'.fts')
    else:
        hdulist = pyfits.open(file_date+str("_")+str(n)+'.fts')

    data=np.zeros((90,400))

    for y in range(10,100):
        for x in range(10,410):
            data[y-10,x-10]=-np.log((hdulist[0].data[0,y,x])/(hdulist[0].data[1,y,x]))

    hdulist.close()
    return data-bg

def fitsopen_bg(n,bg):
    hdulist = pyfits.open(file_date+str("_")+str(n).zfill(2)+'.fts')
    images = hdulist[0].data
    no_absorb = images[0]
    absorb = images[1]
    div = (no_absorb-bg)/(absorb-bg)
    
    div = div[slicer]
    div = -np.log(div)
    div = np.nan_to_num(div)
    return div

def twoD_Gaussian(xy_mesh, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    (x, y) = xy_mesh
    xo = float(xo)
    yo = float(yo)
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo)
                        + c*((y-yo)**2)))
    return g.flatten()


def two_Gaussian(xy_mesh, amplitude1, xo1, yo1, sigma_x1, sigma_y1, theta1, amplitude2, xo2, yo2, sigma_x2, sigma_y2, theta2, offset):
    (x, y) = xy_mesh
    xo1 = float(xo1)
    yo1 = float(yo1)
    xo2 = float(xo2)
    yo2 = float(yo2)

    a1 = (np.cos(theta1)**2)/(2*sigma_x1**2) + (np.sin(theta1)**2)/(2*sigma_y1**2)
    b1 = -(np.sin(2*theta1))/(4*sigma_x1**2) + (np.sin(2*theta1))/(4*sigma_y1**2)
    c1 = (np.sin(theta1)**2)/(2*sigma_x1**2) + (np.cos(theta1)**2)/(2*sigma_y1**2)
    a2 = (np.cos(theta2)**2)/(2*sigma_x2**2) + (np.sin(theta2)**2)/(2*sigma_y2**2)
    b2 = -(np.sin(2*theta2))/(4*sigma_x2**2) + (np.sin(2*theta2))/(4*sigma_y2**2)
    c2 = (np.sin(theta2)**2)/(2*sigma_x2**2) + (np.cos(theta2)**2)/(2*sigma_y2**2)

    g = offset + amplitude1*np.exp( - (a1*((x-xo1)**2) + 2*b1*(x-xo1)*(y-yo1) + c1*((y-yo1)**2))) + amplitude2*np.exp( - (a2*((x-xo2)**2) + 2*b2*(x-xo2)*(y-yo2) + c2*((y-yo2)**2)))

    return g.flatten()


def make_background(N):
    list_bg=list()
    for n in range(0,N):
        hdulist = pyfits.open(file_date+str("_")+str(n).zfill(2)+'.fts')
        list_bg.append(hdulist[0].data[2])
        hdulist.close()
    bg_mean = np.array(list_bg).mean(axis=0)
    bg_std = np.array(list_bg).std(axis=0)
    return bg_mean,bg_std
        



def LZ(x,amplitude,W,x0,c):
    return amplitude*np.exp(2*np.pi*W**2/(x-x0))+c

In [3]:
results=list()

model_twoG = Model(two_Gaussian)

params = Parameters()

params = model_twoG.make_params()


'''
Name           Value      Min      Max   Stderr     Vary     Expr Brute_Step
amplitude1    0.3267     -inf      inf     None     True     None     None
amplitude2  -0.06324     -inf      inf     None     True     None     None
offset       0.01289     -inf      inf     None     True     None     None
sigma_x1       43.08       30      200     None     True     None     None
sigma_x2       5.728        3       15     None     True     None     None
sigma_y1       78.89       30      200     None     True     None     None
sigma_y2       5.728     -inf      inf     None    False sigma_x2     None
theta1      3.378e-09        0    3.142     None     True     None     None
theta2             0        0    3.142     None     True     None     None
xo1            39.46       30       60     None     True     None     None
xo2            46.32       46       50     None     True     None     None
yo1            150.5      140      180     None     True     None     None
yo2            170.1      168      175     None     True     None     None
'''

# cloud distribution
params.add('amplitude1',value=0.1)
params.add('yo1',value=150*binning_scale,min=100*binning_scale,max=200*binning_scale)
params.add('xo1',value=40*binning_scale,min=20*binning_scale,max=60*binning_scale)
params.add('sigma_x1',value=43*binning_scale,min=20*binning_scale,max=100*binning_scale)
params.add('sigma_y1',value=78*binning_scale,min=20*binning_scale,max=100*binning_scale)
params.add('theta1',value=0,min=0,max=np.pi/50)

# EIT/Autler-Townes-dip
params.add('amplitude2',value=0.00)
params.add('yo2',value=170*binning_scale,min=168*binning_scale,max=175*binning_scale)
params.add('xo2',value=46*binning_scale,min=46*binning_scale,max=50*binning_scale)
params.add('sigma_x2',value=10*binning_scale,min=2*binning_scale,max=15*binning_scale)
#params.add('sigma_y2',value=10,min=3,max=15)
params.add('sigma_y2',expr='sigma_x2')
params.add('theta2',value=0,min=0,max=np.pi,vary=False)

# offset
params.add('offset',value=0,vary=False)
bg_mean,bg_std = make_background(N)

model=model_twoG


for n in range(0,len(variables)):
    bg,std= make_background(N)
    image = fitsopen_bg(n,bg)
    image =  gaussian_filter(image, 0.6, order=0, output=None, mode='nearest', cval=0.0, truncate=3.0)
    shape = image.shape
    x,y = np.mgrid[0:shape[0],0:shape[1]]
    image_flat=image.flatten()    
    out = model.fit(image_flat,params,xy_mesh=(x,y))##method='Powel')
    fig,ax = plt.subplots(3,1,figsize=(15,15))
    results.append(copy.deepcopy(out))
    #params = out.params
    #out.params.pretty_print()
    vmax = 0.5
    ax[0].set_title('Image Number '+str(n))
    ax[0].imshow(image, origin='bottom',vmin=0, vmax=vmax)
    ax[1].set_title('Fit')
    ax[1].imshow(out.best_fit.reshape(shape),origin='bottom',vmin=0, vmax=vmax)
    ax[2].set_title('Residual')
    ax[2].imshow((image-out.best_fit.reshape(shape))/out.best_fit.reshape(shape),origin='bottom',vmin=-1, vmax=1, cmap='coolwarm')
    plt.show()